In [126]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
import requests
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [127]:
from selenium.webdriver.common.by import By

In [129]:
# driver = webdriver.Chrome()
# driver.get("https://www.glassdoor.com/Job/united-states-data-scientist-jobs-SRCH_IL.0,13_IN1_KO14,28_IP7.htm?includeNoSalaryJobs=true")

#     # Get element with tag name 'div'
# element = driver.find_element(By.CSS_SELECTOR, '[class="SVGInline modal_closeIcon"]').click()

# driver.maximize_window()

In [130]:
jobs = []

In [146]:
driver = webdriver.Chrome()
# driver.get("https://www.glassdoor.com/Job/united-states-data-scientist-jobs-SRCH_IL.0,13_IN1_KO14,28_IP7.htm?includeNoSalaryJobs=false")

num_jobs = 900
verbose = True
dup = 0
driver.get("https://www.glassdoor.com/Job/us-data-engineer-jobs-SRCH_IL.0,2_IN1_KO3,16.htm?clickSource=searchBox")
# driver.get("https://www.glassdoor.com/Job/data-scientist-jobs-SRCH_KO0,14.htm?clickSource=searchBox")

while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.
        
        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(2)

        #Test for the "Sign Up" prompt and get rid of it.
#         try:
#             driver.find_element(By.CSS_SELECTOR, '[class="SVGInline modal_closeIcon"]').click()
#         except ElementClickInterceptedException:
#             pass

#         time.sleep(.1)

        try:
            driver.find_element(By.CSS_SELECTOR, '[class="SVGInline modal_closeIcon"]').click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements(By.CSS_SELECTOR, '[data-test="jobListing"]')  #jl for Job Listing. These are the buttons we're going to click.
        
              
        for job_button in job_buttons:  
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs) + "/ dup" + str(dup)))
            
            if len(jobs) >= num_jobs:
                break
                
            try:
                driver.find_element(By.CSS_SELECTOR, '[alt="Close"]')
            except:
                try:
                    driver.find_element(By.CSS_SELECTOR, '[alt="Close"]').click()  #clicking to the X.
                except NoSuchElementException:
                    pass
            
#             job_button.click()
            
            try:
                job_button.click()
            except NoSuchElementException:
                print(job_button.text)

      
            try:
                if driver.find_element(By.CSS_SELECTOR, '[alt="Close"]'):
                    time.sleep(3)
                    driver.find_element(By.CSS_SELECTOR, '[alt="Close"]').click()  #clicking to the X.
            except NoSuchElementException:
                pass

            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    try:
                        driver.find_element(By.CSS_SELECTOR, '[data-test="employerName"]')
                        company_name = driver.find_element(By.CSS_SELECTOR, '[data-test="employerName"]').text
                    except: 
                        company_name = driver.find_element(By.CSS_SELECTOR, '[class="d-flex justify-content-between"]').text
#                     try:
#                         company_name = driver.find_element(By.CSS_SELECTOR, '[class="d-flex justify-content-between"]').text
#                     except:
#                         NoSuchElementException
                    location = driver.find_element(By.CSS_SELECTOR, '[data-test="location"]').text
                    job_title = driver.find_element(By.CSS_SELECTOR, '[class="css-1vg6q84 e1tk4kwz4"]').text
                    driver.find_element(By.CSS_SELECTOR, '[class="css-t3xrds e856ufb4"]').click()
                    # time.sleep(10)
                    job_description = driver.find_element(By.CSS_SELECTOR, '[id="JobDescriptionContainer"]').text
                    # salary_estimate = driver.find_element(By.CSS_SELECTOR, '[class="css-1xe2xww e1wijj242"]').text
                    collected_successfully = True
#                     if len(job_title)<1:
#                         time.sleep(5)
#                         job_title = driver.find_element(By.CSS_SELECTOR, '[data-test="jobTitle"]').text
                except:
                    time.sleep(5)
                    
                    
            # class="d-flex justify-content-between"

            try:
#                 salary_element = WebDriverWait(driver, 10).until(
#                                     EC.presence_of_element_located((By.CSS_SELECTOR, "[class='css-1bluz6i e2u4hf13']")))
                salary_estimate = driver.find_element(By.CSS_SELECTOR, '[class="css-1bluz6i e2u4hf13"]').get_attribute("textContent")
#                 salary_estimate = salary_element.text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element(By.CSS_SELECTOR, '[data-test="detailRating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500])) #[:500]
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            
            #try:
            overview_name = driver.find_elements(By.CSS_SELECTOR, '[class="css-1taruhi e1pvx6aw1"]')
            overview_name_list = [element.text for element in overview_name]
            overview = driver.find_elements(By.CSS_SELECTOR, '[class="css-i9gxme e1pvx6aw2"]')
            overview_list = [element.text for element in overview]
#             for name, value in zip(overview_name_list, overview_list):
            if "Size" in overview_name_list:
                size = overview_list[overview_name_list.index("Size")]
            else:
                size = -1

            if "Founded" in overview_name_list:
                founded = overview_list[overview_name_list.index("Founded")]
            else:
                founded = -1

            if "Type" in overview_name_list:
                type_of_ownership = overview_list[overview_name_list.index("Type")]
            else:
                type_of_ownership = -1

            if "Industry" in overview_name_list:
                industry = overview_list[overview_name_list.index("Industry")]
            else:
                industry = -1

            if "Sector" in overview_name_list:
                sector = overview_list[overview_name_list.index("Sector")]
            else:
                sector = -1

            if "Revenue" in overview_name_list:
                revenue = overview_list[overview_name_list.index("Revenue")]
            else:
                revenue = -1

#                     founded = overview[1].text
#                     type_of_ownership = overview[2].text
#                     industry = overview[3].text
#                     sector = overview[4].text
#                     revenue = overview[5].text
#                 else:# NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
#                     headquarters = -1
#                     size = -1
#                     founded = -1
#                     type_of_ownership = -1
#                     industry = -1
#                     sector = -1
#                     revenue = -1

                
            if verbose:
                # print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                # print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
            new_dict = {"Job Title" : job_title,
                        "Salary Estimate" : salary_estimate,
                        "Job Description" : job_description,
                        "Rating" : rating,
                        "Company Name" : company_name,
                        "Location" : location,
                        #"Headquarters" : headquarters,
                        "Size" : size,
                        "Founded" : founded,
                        "Type of ownership" : type_of_ownership,
                        "Industry" : industry,
                        "Sector" : sector,
                        "Revenue" : revenue,}
            if new_dict not in jobs:
                jobs.append(new_dict)
            else:
                dup += 1
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element(By.CSS_SELECTOR, '[aria-label="Next"]').click()
            if dup > 200:
                break
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

Progress: 206/900/ dup0
Job Title: Sr. Data Engineer
Salary Estimate: $98,486 /yr (est.)
Job Description: Overview:
We hire people, not positions. That's because at Genesis, we put people first, and that includes our customers, partners, and employees.

Genesis is the national leader in non-prime credit card. We serve millions of customers, and we're fair and transparent in all interactions.

As a company grounded in entrepreneurship, we're looking to expand our team and are looking for people who foster innovation and strive to make an impact.

We’re an established company with over 20 years of exp
Rating: 3.0
Company Name: Genesis Financial Solutions, Inc.
3.0
Location: Beaverton, OR
Size: 501 to 1000 Employees
Founded: 2001
Type of Ownership: Company - Private
Industry: Banking & Lending
Sector: Financial Services
Revenue: $500 million to $1 billion (USD)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 207/900/ dup0
Job Title: Senior Engineer, Data Analytics
Salary Es

Job Title: Research Engineer (Software/Data Science)
Salary Estimate: $80,000 /yr (est.)
Job Description: Description
VoltServer is seeking a Research Engineer to join our growing R&D team. We are seeking an individual with research experience that can help with a variety of special projects which may include data analysis, simulation design, brainstorming new ideas, reviewing technical publications, supporting technical issues, etc. Your exact background is not critical, but you will need a good ability to think critically and analyze technical data in order to be successful in this role. You shoul
Rating: -1
Company Name: Voltserver
Location: East Greenwich, RI
Size: 1 to 50 Employees
Founded: -1
Type of Ownership: Company - Public
Industry: -1
Sector: -1
Revenue: Unknown / Non-Applicable
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 215/900/ dup1
Job Title: Data Link Engineer
Salary Estimate: $110,000 /yr (est.)
Job Description: Candidate will need basic understandi

Job Title: Senior Data Platform Engineer
Salary Estimate: $111,922 /yr (est.)
Job Description: :
The Data Platform Engineer is a key resource in providing technical leadership and support on all the stages of Data Platform administration, development, and architecture implementation. We’re looking for an enthusiastic team member to engage and collaborate with leaders and team members from teams across our enterprise (including Server, Infrastructure, dozens of Development teams, and more), as we continue our transformation from a platform-based to a solution-based shop. This team member w
Rating: 2.7
Company Name: United Wholesale Mortgage
2.7
Location: Pontiac, MI
Size: 5001 to 10000 Employees
Founded: 1986
Type of Ownership: Company - Public
Industry: Banking & Lending
Sector: Financial Services
Revenue: Unknown / Non-Applicable
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 224/900/ dup2
Job Title: Data Engineer
Salary Estimate: $80,414 /yr (est.)
Job Description: Ov

Size: 51 to 200 Employees
Founded: 1999
Type of Ownership: Company - Private
Industry: Enterprise Software & Network Solutions
Sector: Information Technology
Revenue: $5 to $25 million (USD)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 232/900/ dup3
Job Title: Data Platform Engineer File Transfer
Salary Estimate: $106,038 /yr (est.)
Job Description: RESPONSIBILITIES:
Design, build, integrate, and maintain data platforms critical to the Sanofi Digital Data Team.
Must work with business and technical teams to understand requirements and translate them into technical needs to enable and expand our data products and services.
Partner with Product Owner, Data Architects and other Data Engineers to prioritize the data platform implementation plan
Ensure automation best practices are a foundational component in building data platforms.
Actively co
Rating: 4.0
Company Name: Sanofi
4.0
Location: Bridgewater, NJ
Size: 10000+ Employees
Founded: 1973
Type of Ownership: Company - 

Job Title: Data Engineer
Salary Estimate: $87,500 /yr (est.)
Job Description: Job description
About the Role:Data and translation of data into information is core to Trailhead Biosystems strategy and business. You will work directly with our founding team to help develop a backend data infrastructure to both support Trailhead Biosystems internal bioinformaticians and platform as well as external-facing partnerships. You will have the opportunity to evaluate cutting edge data tools and be part of a team working in the space of computational biology.The ideal candidate will
Rating: 2.5
Company Name: Trailhead Biosystems Inc.
2.5
Location: Beachwood, OH
Size: 1 to 50 Employees
Founded: -1
Type of Ownership: Unknown
Industry: -1
Sector: -1
Revenue: Unknown / Non-Applicable
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 242/900/ dup3
Job Title: Data Acquisition Engineer (Night Shift)
Salary Estimate: $61,482 /yr (est.)
Job Description: Data Acquisition Engineer (Night Shift)

...

Size: 10000+ Employees
Founded: 1853
Type of Ownership: Company - Public
Industry: Insurance Carriers
Sector: Insurance
Revenue: $10+ billion (USD)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 371/900/ dup180
Job Title: Sr Data Storage Engineer, remote
Salary Estimate: $102,499 /yr (est.)
Job Description: Overview:
Fairview Health Services is hiring a Sr Data Storage Engineer! The Sr Engineer role includes crafting, configuring, developing, integrating, testing, implementing, detailing and supporting data storage infrastructure environments. Our Storage Services team handles the building of services through a deep understanding of the application requirements. We work with Architecture and Operational teams to develop, apply and support Enterprise Storage and Backup Infrastructure processes, tool
Rating: 3.4
Company Name: Fairview Health Services
3.4
Location: Minneapolis, MN
Size: 10000+ Employees
Founded: -1
Type of Ownership: Nonprofit Organization
Industry: Health

Job Title: Sr Data Integration Engineer
Salary Estimate: $113,745 /yr (est.)
Job Description: Compeer Financial is seeking collaborative, innovative and dynamic professionals to be a part of our Top Workplace culture!
Tell me more about this opportunity.
Position Overview:
This position is responsible for the development, enhancement and effective operations of data movement between various systems using ETL methodologies. Responsible for leading company database development efforts and coordinating with business users, consultants/contractors, vendors and internal team members to maximi
Rating: 4.4
Company Name: Compeer Financial
4.4
Location: Bloomington, IL
Size: 1001 to 5000 Employees
Founded: 1916
Type of Ownership: Company - Private
Industry: Banking & Lending
Sector: Financial Services
Revenue: $10+ billion (USD)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 372/900/ dup189
Job Title: ETL Data Engineer
Salary Estimate: $98,800 /yr (est.)
Job Description: Overvie

Job Title: Cloud Data Engineer I - AWS, Python, Snowflake, SQL
Salary Estimate: $135,900 /yr (est.)
Job Description: Who Are We?
Taking care of our customers, our communities and each other. That’s the Travelers Promise. By honoring this commitment, we have maintained our reputation as one of the best property casualty insurers in the industry for over 160 years. Join us to discover a culture that is rooted in innovation and thrives on collaboration. Imagine loving what you do and where you do it.
Compensation Overview
The annual base salary range provided for this position is a nationwide market range and rep
Rating: 4.0
Company Name: The Travelers Companies, Inc.
4.0
Location: Hartford, CT
Size: 10000+ Employees
Founded: 1853
Type of Ownership: Company - Public
Industry: Insurance Carriers
Sector: Insurance
Revenue: $10+ billion (USD)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 372/900/ dup198
Job Title: Senior Data Engineer with Top Secret
Salary Estimate: $103,55

Size: 5001 to 10000 Employees
Founded: 1980
Type of Ownership: Company - Private
Industry: Wholesale
Sector: Retail & Wholesale
Revenue: $5 to $10 billion (USD)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 372/900/ dup207
Job Title: Sr Data Storage Engineer, remote
Salary Estimate: $102,499 /yr (est.)
Job Description: Overview:
Fairview Health Services is hiring a Sr Data Storage Engineer! The Sr Engineer role includes crafting, configuring, developing, integrating, testing, implementing, detailing and supporting data storage infrastructure environments. Our Storage Services team handles the building of services through a deep understanding of the application requirements. We work with Architecture and Operational teams to develop, apply and support Enterprise Storage and Backup Infrastructure processes, tool
Rating: 3.4
Company Name: Fairview Health Services
3.4
Location: Minneapolis, MN
Size: 10000+ Employees
Founded: -1
Type of Ownership: Nonprofit Organization
Ind

Size: 10000+ Employees
Founded: 1933
Type of Ownership: Self-employed
Industry: Banking & Lending
Sector: Financial Services
Revenue: Unknown / Non-Applicable
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 373/900/ dup215
Job Title: ETL Data Engineer
Salary Estimate: $98,000 /yr (est.)
Job Description: CDPHP and its family of companies are mission-driven organizations that support the health and well-being of our customers and the communities we are proud to serve. CDPHP was founded in Albany in 1984 as a physician-guided not-for-profit, and currently offers health plans in 29 counties in New York state. The company values integrity, diversity, and innovation, and its corporate culture supports those values wholeheartedly. At CDPHP, the employees have a voice and are encouraged to make an impa
Rating: 4.4
Company Name: CAPITAL DISTRICT PHYSICIANS HEALTH PLAN INC
4.4
Location: Albany, NY
Size: 501 to 1000 Employees
Founded: 1984
Type of Ownership: Nonprofit Organization


In [147]:
first_scrape = jobs
first_scrape

[{'Job Title': 'Data Scientist V',
  'Salary Estimate': '$111,388\xa0/yr (est.)',
  'Job Description': 'Posting End Date:\nApril 09, 2023\nEmployee Type:\nRegular-Full time\nUnion/Non:\nThis is a non-union position\nLife takes energy. The Enbridge Technology + Innovation Lab works with data that powers our products to improve safety and reliability. By working hands-on with ground-breaking technology, the lab pioneers the development of innovative products through small agile teams. Our teams incorporate a variety of multidisciplinary skills, including industrial predictive algorithms, machine learning, and sentiment analysis.\nAs a Data Scientist V, you’ll bring a deep understanding of big data and will help the T+IL to build and enable big data analytics solutions. We build advanced technology products to deliver energy in a safe and reliable way to fuel a rapidly innovating world. Here in our new Technology + Innovation Lab we believe in Agile way of working, focus on our people and

In [148]:
import pandas as pd
stored_dataframe = pd.read_csv('glassdoor_ds_job_salary.csv')

In [149]:
newly_scraped = pd.DataFrame(first_scrape)
newly_scraped.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Size', 'Founded', 'Type of ownership',
       'Industry', 'Sector', 'Revenue'],
      dtype='object')

In [150]:
new_df = pd.concat([stored_dataframe, newly_scraped], axis=0)
new_df.drop_duplicates()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist V,"$111,362 /yr (est.)","Posting End Date:\nApril 09, 2023\nEmployee Ty...",4.1,Enbridge\n4.1,"Houston, TX",10000+ Employees,1949,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",$10+ billion (USD)
1,Data Scientist,-1,Data Scientist\nJob Category: Science\nTime Ty...,4.0,CACI\n4.0,"Charlottesville, VA",10000+ Employees,1962,Company - Public,Aerospace & Defense,Aerospace & Defense,$1 to $5 billion (USD)
2,Designer UIX NLP/NLU,"$117,500 /yr (est.)",* W2 Only *\nThe successful candidate will pro...,-1.0,"RightFit Talent Brokers, LLC",Remote,-1,-1,-1,-1,-1,-1
3,Machine Learning Engineer,"$129,337 /yr (est.)",Signal Systems Corporation (SSC) is searching ...,-1.0,Signal Systems Corporation,"Millersville, MD",-1,-1,-1,-1,-1,-1
4,Senior Quality Data Analyst,"$80,968 /yr (est.)","FIRST SHIFT - Draeger Medical Systems, Inc. - ...",3.8,"Draeger Medical Systems, Inc.\n3.8","Andover, MA",10000+ Employees,1889,Company - Public,Health Care Products Manufacturing,Manufacturing,$1 to $5 billion (USD)
...,...,...,...,...,...,...,...,...,...,...,...,...
369,Data Engineer,"$82,200 /yr (est.)",The Opportunity\nDelaware North Global Headqua...,3.5,Delaware North\n3.5,"Buffalo, NY",10000+ Employees,1915,Company - Private,Catering & Food Service Contractors,Restaurants & Food Service,$1 to $5 billion (USD)
370,Senior Data Engineer,"$110,500 /yr (est.)",Company Description\n\nPublicis Sapient is a d...,3.9,Publicis Sapient\n3.9,"Houston, TX",10000+ Employees,1990,Company - Public,Business Consulting,Management & Consulting,Unknown / Non-Applicable
371,"Sr Data Storage Engineer, remote","$102,499 /yr (est.)",Overview:\nFairview Health Services is hiring ...,3.4,Fairview Health Services\n3.4,"Minneapolis, MN",10000+ Employees,-1,Nonprofit Organization,Health Care Services & Hospitals,Healthcare,$1 to $5 billion (USD)
372,Data Center Systems Engineer - TS/SCI with Poly,"$150,000 /yr (est.)",Description\nJob Description:\nThe Integrated ...,3.9,Leidos\n3.9,"Springfield, VA",10000+ Employees,1969,Company - Public,Aerospace & Defense,Aerospace & Defense,$10+ billion (USD)


In [152]:
new_df.drop_duplicates(subset=['Job Title', 'Salary Estimate', 'Rating',
       'Company Name', 'Location', 'Size', 'Founded', 'Type of ownership',
       'Industry', 'Sector', 'Revenue'], keep='last')

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist V,"$111,362 /yr (est.)","Posting End Date:\nApril 09, 2023\nEmployee Ty...",4.1,Enbridge\n4.1,"Houston, TX",10000+ Employees,1949,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",$10+ billion (USD)
1,Data Scientist,-1,Data Scientist\nJob Category: Science\nTime Ty...,4.0,CACI\n4.0,"Charlottesville, VA",10000+ Employees,1962,Company - Public,Aerospace & Defense,Aerospace & Defense,$1 to $5 billion (USD)
2,Designer UIX NLP/NLU,"$117,500 /yr (est.)",* W2 Only *\nThe successful candidate will pro...,-1.0,"RightFit Talent Brokers, LLC",Remote,-1,-1,-1,-1,-1,-1
3,Machine Learning Engineer,"$129,337 /yr (est.)",Signal Systems Corporation (SSC) is searching ...,-1.0,Signal Systems Corporation,"Millersville, MD",-1,-1,-1,-1,-1,-1
4,Senior Quality Data Analyst,"$80,968 /yr (est.)","FIRST SHIFT - Draeger Medical Systems, Inc. - ...",3.8,"Draeger Medical Systems, Inc.\n3.8","Andover, MA",10000+ Employees,1889,Company - Public,Health Care Products Manufacturing,Manufacturing,$1 to $5 billion (USD)
...,...,...,...,...,...,...,...,...,...,...,...,...
369,Data Engineer,"$82,200 /yr (est.)",The Opportunity\nDelaware North Global Headqua...,3.5,Delaware North\n3.5,"Buffalo, NY",10000+ Employees,1915,Company - Private,Catering & Food Service Contractors,Restaurants & Food Service,$1 to $5 billion (USD)
370,Senior Data Engineer,"$110,500 /yr (est.)",Company Description\n\nPublicis Sapient is a d...,3.9,Publicis Sapient\n3.9,"Houston, TX",10000+ Employees,1990,Company - Public,Business Consulting,Management & Consulting,Unknown / Non-Applicable
371,"Sr Data Storage Engineer, remote","$102,499 /yr (est.)",Overview:\nFairview Health Services is hiring ...,3.4,Fairview Health Services\n3.4,"Minneapolis, MN",10000+ Employees,-1,Nonprofit Organization,Health Care Services & Hospitals,Healthcare,$1 to $5 billion (USD)
372,Data Center Systems Engineer - TS/SCI with Poly,"$150,000 /yr (est.)",Description\nJob Description:\nThe Integrated ...,3.9,Leidos\n3.9,"Springfield, VA",10000+ Employees,1969,Company - Public,Aerospace & Defense,Aerospace & Defense,$10+ billion (USD)


In [ ]:
new_df[new_df.duplicated(subset=['Job Title', 'Salary Estimate', 'Rating',
       'Company Name', 'Location', 'Size', 'Founded', 'Type of ownership',
       'Industry', 'Sector', 'Revenue'])].sort_values('Company Name')

In [144]:
updated_scraped = new_df.drop_duplicates(subset=['Job Title', 'Salary Estimate', 'Rating',
       'Company Name', 'Location', 'Size', 'Founded', 'Type of ownership',
       'Industry', 'Sector', 'Revenue'], keep='last')

In [145]:
updated_scraped.to_csv('glassdoor_ds_job_salary.csv', index=False)

In [15]:
stored_dataframe

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist V,"$111,362 /yr (est.)","Posting End Date:\nApril 09, 2023\nEmployee Ty...",4.1,Enbridge\n4.1,"Houston, TX",10000+ Employees,1949,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",$10+ billion (USD)
1,Data Scientist,-1,Data Scientist\nJob Category: Science\nTime Ty...,4.0,CACI\n4.0,"Charlottesville, VA",10000+ Employees,1962,Company - Public,Aerospace & Defense,Aerospace & Defense,$1 to $5 billion (USD)
2,Data Science/Analysis Research Librarian,-1,Hamilton College invites applications for the ...,4.6,Hamilton College\n4.6,"Clinton, NY",501 to 1000 Employees,1812,College / University,Colleges & Universities,Education,$25 to $100 million (USD)
3,Senior Systems Engineer - Systems Modeling & C...,"$125,000 /yr (est.)",You will have an opportunity to directly impac...,4.1,Cenergy International Services\n4.1,"Boston, MA",10000+ Employees,1907,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",$10+ billion (USD)
4,Senior Data Scientist - TS Clearance Required,"$131,400 /yr (est.)",Sr Data Scientist (up to 25% profit sharing be...,-1.0,Eighty20 Collective,"Sterling, VA",Unknown,-1,Company - Public,-1,-1,Unknown / Non-Applicable
...,...,...,...,...,...,...,...,...,...,...,...,...
277,Data Scientist - Supply Planning,"$100,638 /yr (est.)",Beam Suntory is Crafting the Spirits that Stir...,3.9,Beam Inc.\n3.9,NaN,5001 to 10000 Employees,2014,Company - Private,Food & Beverage Manufacturing,Manufacturing,$1 to $5 billion (USD)
278,Applied Machine Learning Scientist - Data Tech...,"$205,000 /yr (est.)",Who we are:\nThe Bloomberg Engineering Data Te...,4.1,Bloomberg\n4.1,"Middletown, CT",10000+ Employees,1981,Company - Private,Computer Hardware Development,Information Technology,Unknown / Non-Applicable
279,Senior Data Analyst,$72.50 /hr (est.),Salary: $65-$80/hour\nIndustry: Footwear & App...,4.8,BolyːWelch\n4.8,"Beaverton, OR",1 to 50 Employees,1986,Company - Private,HR Consulting,Human Resources & Staffing,$5 to $25 million (USD)
280,Senior Language Modeling Engineer - Artificial...,"$212,500 /yr (est.)",Bloomberg is a global leader in business and f...,4.1,Bloomberg\n4.1,"Middletown, CT",10000+ Employees,1981,Company - Private,Computer Hardware Development,Information Technology,Unknown / Non-Applicable


In [150]:
second_scrape = jobs
second_scrape

[{'Job Title': 'Data Scientist V',
  'Salary Estimate': '$111,362\xa0/yr (est.)',
  'Job Description': 'Posting End Date:\nApril 09, 2023\nEmployee Type:\nRegular-Full time\nUnion/Non:\nThis is a non-union position\nLife takes energy. The Enbridge Technology + Innovation Lab works with data that powers our products to improve safety and reliability. By working hands-on with ground-breaking technology, the lab pioneers the development of innovative products through small agile teams. Our teams incorporate a variety of multidisciplinary skills, including industrial predictive algorithms, machine learning, and sentiment analysis.\nAs a Data Scientist V, you’ll bring a deep understanding of big data and will help the T+IL to build and enable big data analytics solutions. We build advanced technology products to deliver energy in a safe and reliable way to fuel a rapidly innovating world. Here in our new Technology + Innovation Lab we believe in Agile way of working, focus on our people and

In [171]:
combined_first_second = first_scrape + second_scrape + third_scrape + success_list
len(combined_first_second)

1458

In [165]:
third_scrape = jobs
len(third_scrape)

101

In [209]:
import json
unique_values = set(json.dumps(d) for d in success_list)

print(len(unique_values))
unique_values

146


{'{"Job Title": "AI Artificial Intelligence Developer", "Salary Estimate": "$103,572\\u00a0/yr (est.)", "Job Description": "Genesis10 is seeking an Application Developer for a 12 month contract to hire position with a corporate investment banking company located in Jersey City, NJ. This is a hybrid position and requires being on site 3 days per week.\\n\\nSummary:\\nThis developer is in the conversational artificial intelligence space. As a Conversational AI Lead, you will join our Cognitive Automation team. You will be responsible for every stage of a Conversational AI project - lead the architecture, development, and implementation of conversational AI projects from research and design to conversation scripting and testing.\\n\\nResponsibilities:\\nOwn every stage of a Conversational AI project - lead the architecture, development, and implementation of conversational AI projects from research and design to conversation scripting and testing.\\n\\n\\n\\n\\n\\n\\n\\nRanked a Top Staff

In [174]:
success_list = jobs
len(success_list)
success_set = set(json.dumps(d) for d in success_list)
len(success_set)

146

In [123]:
from collections import defaultdict
def frequencies(lst):
    freq = defaultdict(int)
    for val in lst:
        freq[val] += 1
    return dict(freq)



In [125]:
list = [1, 2, 3]
frequencies(list)

{1: 1, 2: 1, 3: 1}


In [189]:
success_lst = [json.dumps(d) for d in success_list]

In [195]:
successs_dict = frequencies(success_lst)

In [199]:
type(successs_dict)

dict

In [200]:
for key, value in successs_dict.items():
    print(value)

25
25
25
22
25
25
25
12
25
25
24
11
25
25
25
25
25
19
25
5
2
25
25
12
25
1
1
24
3
1
16
1
12
7
1
5
21
8
5
12
9
2
2
16
2
6
14
4
2
13
4
1
2
2
2
2
1
1
2
1
2
5
12
2
1
1
2
1
1
2
1
2
2
1
1
1
1
1
1
2
10
1
2
1
1
3
1
3
1
1
2
3
1
2
2
2
2
1
1
1
2
1
1
2
1
1
1
2
1
2
2
2
1
2
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
2
17
1
3
1
3
2
2
1
1


In [202]:
successs_dict

{'{"Job Title": "Senior Python Data Scientist", "Salary Estimate": -1, "Job Description": "Overview:\\nOasis Systems has an exciting opportunity for a Senior Python Data Scientist to support the Commodity Futures Trading Commission (CFTC) in Washington, DC. The contractor shall provide support in data analytics (including descriptive and predictive analytics and data mining), large data manipulation, data visualization, and the use of data analytic tools. Activities that the contractor may be tasked to perform include gathering requirements from stakeholders, conducting financial data analysis in a Big Data environment, and creating reports and documentation.\\n\\nLOCATION: Washington, DC\\nJOB STATUS: Full-Time\\nTRAVEL: N/A\\nREQUIRED QUALIFICATIONS (Education, Certifications, Experience, Skills)\\nSECURITY CLEARANCE: Ability to pass a Special Agreement Check (SAC) and a National Agency Check and Inquiries (NACI) Background Investigation and U.S. Citizenship is required for all appli

In [ ]:
first_scrape.extend(second_scrape)

In [59]:
response = requests.get(target_url)
if response.status_code == 200:
    print("Success")
else:
    print("Failure")
soup = BeautifulSoup(response.content, 'html.parser')
allJobsContainer = soup.find("ul")

#allJobs = allJobsContainer.find_all("li")

Failure


In [60]:
response.status_code

403

In [57]:
allJobsContainer

In [134]:
driver = webdriver.Chrome()
driver.get("https://www.glassdoor.com/Job/united-states-data-scientist-jobs-SRCH_IL.0,13_IN1_KO14,28_IP2.htm?includeNoSalaryJobs=true&pgc=AB4AAYEAHgAAAAAAAAAAAAAAAf8xnJwAQwEBAQcCAHscnY62Qk942mPGLgWfOJanLoCN0GWcjSMiOGPasDuVRUW2qPToi7eE38gBBUcX%2B491ifLlRBwp%2BqUxcl8AAA%3D%3D")

    # Get element with tag name 'div'
element = driver.find_element(By.CSS_SELECTOR, '[alt="Close"]').click()
job_buttons = driver.find_elements(By.CSS_SELECTOR, '[data-test="jobListing"]')  #jl for Job Listing. These are the buttons we're going to click.
for job_button in job_buttons: 
    try:
        driver.find_element(By.CSS_SELECTOR, '[alt="Close"]').click()  #clicking to the X.
    except NoSuchElementException:
        pass
    job_button.click()
    try:
        driver.find_element(By.CSS_SELECTOR, '[alt="Close"]').click()  #clicking to the X.
    except NoSuchElementException:
        pass

            
    collected_successfully = False

    while not collected_successfully:
        try:
            try:
                company_name = driver.find_element(By.CSS_SELECTOR, '[data-test="employerName"]').text
            except: 
                company_name = driver.find_element(By.CSS_SELECTOR, '[class="d-flex justify-content-between"]').text
            company_name = driver.find_element(By.CSS_SELECTOR, '[data-test="employerName"]').text
            location = driver.find_element(By.CSS_SELECTOR, '[data-test="location"]').text
            job_title = driver.find_element(By.CSS_SELECTOR, '[data-test="job-link"]').text
            driver.find_element(By.CSS_SELECTOR, '[class="css-t3xrds e856ufb4"]').click()
            job_description = driver.find_element(By.CSS_SELECTOR, '[id="JobDescriptionContainer"]').text
            collected_successfully = True
        except:
            time.sleep(5)

        try:
            salary_estimate = driver.find_element(By.CSS_SELECTOR, '[data-test="detailSalary"]').text
        except NoSuchElementException:
            salary_estimate = -1 #You need to set a "not found value. It's important."

        try:
            rating = driver.find_element(By.CSS_SELECTOR, '[data-test="detailRating"]').text
        except NoSuchElementException:
            rating = -1 #You need to set a "not found value. It's important."

    overview = driver.find_elements(By.CSS_SELECTOR, '[class="css-i9gxme e1pvx6aw2"]')
    break
# #for each in overview:
# size = overview[0].text
# founded = overview[1].text
# type_of_ownership = overview[2].text
# industry = overview[3].text
# sector = overview[4].text
# revenue = overview[5].text
    #print(each.text)
driver.maximize_window()


In [135]:
job_description

"FLSA Status\nExempt\nSalary Range\n$135,000-$145,000 + incentive bonus of up to 20% of base salary\nExact compensation may vary based on skills, experience, and location.\nJob Family\nData Strategy and Services\n\n**Fusion Medical Staffing does not offer Visa Sponsorship at this time**\n\nOverview: Fusion Medical Staffing is searching for a talented Manager of Data Engineering and Science to join our growing Data Strategy and Services team. In this role, you will play a pivotal role for continued design, development and execution of a Centralized Data Warehouse while managing the flow of data and providing direction for deep analysis and leading Fusion’s Data Science Projects. You will be responsible for maintaining and growing an enterprise-wide data infrastructure that will supply the business with high quality, certified data sets and a structure that will support and enable the future of the business for Data Analytics, Machine Learning, AI, and Data Sciences. You will work with a

In [87]:
company_name

'Enbridge\n4.1'

In [88]:
location

'Houston, TX'

In [89]:
job_title

'Data Scientist V'

In [90]:
job_description

'Posting End Date:\nApril 09, 2023\nEmployee Type:\nRegular-Full time\nUnion/Non:\nThis is a non-union position\nLife takes energy. The Enbridge Technology + Innovation Lab works with data that powers our products to improve safety and reliability. By working hands-on with ground-breaking technology, the lab pioneers the development of innovative products through small agile teams. Our teams incorporate a variety of multidisciplinary skills, including industrial predictive algorithms, machine learning, and sentiment analysis.\nAs a Data Scientist V, you’ll bring a deep understanding of big data and will help the T+IL to build and enable big data analytics solutions. We build advanced technology products to deliver energy in a safe and reliable way to fuel a rapidly innovating world. Here in our new Technology + Innovation Lab we believe in Agile way of working, focus on our people and value craftsmanship, in a ‘work is play’ environment where we offer a flexible, fun, and modern workin

In [91]:
salary_estimate

'$94K - $132K (Glassdoor est.)'

In [92]:
rating

'4.1'

In [95]:
size

'10000+ Employees'

In [96]:
founded

'1949'

In [97]:
type_of_ownership

'Company - Public'

In [98]:
industry

'Energy & Utilities'

In [99]:
sector

'Energy, Mining & Utilities'

In [ ]:
revenue

In [108]:
job_buttons

[<selenium.webdriver.remote.webelement.WebElement (session="b3bc03238d283218d55ac7cd65840d5d", element="c8d23342-6b79-46eb-9451-7a0b9839debe")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3bc03238d283218d55ac7cd65840d5d", element="087f8e60-392d-441a-b278-488151be0aba")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3bc03238d283218d55ac7cd65840d5d", element="c909546e-d596-4b36-8a1a-6282bbcc5475")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3bc03238d283218d55ac7cd65840d5d", element="a2102519-b736-4906-b0f7-943039733798")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3bc03238d283218d55ac7cd65840d5d", element="77fb654c-7d41-4537-a0b2-56e5942b948e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3bc03238d283218d55ac7cd65840d5d", element="3e426682-577d-4f78-9345-1b24d393d8e7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3bc03238d283218d55ac7cd65840d5d", element="909a9eab-9409-4c2c-9973-82

In [117]:
type(job_title)

str

In [205]:
import pandas as pd
df = pd.DataFrame(combined_first_second)
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist V,"$111,362 /yr (est.)","Posting End Date:\nApril 09, 2023\nEmployee Ty...",4.1,Enbridge\n4.1,"Houston, TX",10000+ Employees,1949,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",$10+ billion (USD)
1,Data Scientist,-1,Data Scientist\nJob Category: Science\nTime Ty...,4.0,CACI\n4.0,"Charlottesville, VA",10000+ Employees,1962,Company - Public,Aerospace & Defense,Aerospace & Defense,$1 to $5 billion (USD)
2,Data Science/Analysis Research Librarian,-1,Hamilton College invites applications for the ...,4.6,Hamilton College\n4.6,"Clinton, NY",501 to 1000 Employees,1812,College / University,Colleges & Universities,Education,$25 to $100 million (USD)
3,Senior Systems Engineer - Systems Modeling & C...,"$125,000 /yr (est.)",You will have an opportunity to directly impac...,4.1,Cenergy International Services\n4.1,"Boston, MA",10000+ Employees,1907,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",$10+ billion (USD)
4,Senior Data Scientist - TS Clearance Required,"$131,400 /yr (est.)",Sr Data Scientist (up to 25% profit sharing be...,-1,Eighty20 Collective,"Sterling, VA",Unknown,-1,Company - Public,-1,-1,Unknown / Non-Applicable
...,...,...,...,...,...,...,...,...,...,...,...,...
1453,Applied Statistician - Data Technologies,"$205,000 /yr (est.)",Who we are:\nThe Bloomberg Engineering Data Te...,4.1,Bloomberg\n4.1,"New York, NY",10000+ Employees,1981,Company - Private,Computer Hardware Development,Information Technology,Unknown / Non-Applicable
1454,Pipeline Integrity Risk Assessment & Data Scie...,"$148,650 /yr (est.)","About Us\n\nAt SoCalGas, we believe that every...",4.4,SoCalGas\n4.4,"Los Angeles, CA",5001 to 10000 Employees,1929,Subsidiary or Business Segment,Energy & Utilities,"Energy, Mining & Utilities",$1 to $5 billion (USD)
1455,"Data Engineer, Senior Consultant","$97,622 /yr (est.)",Job Family:\nData Science Consulting\n\nTravel...,3.7,Guidehouse\n3.7,"Huntsville, AL",1001 to 5000 Employees,2018,Company - Private,Business Consulting,Management & Consulting,Unknown / Non-Applicable
1456,Senior Decision Science Manager,-1,Position Overview:\nCar Capital Technologies i...,2.4,CAR CAPITAL CORPORATION\n2.4,"Grapevine, TX",1 to 50 Employees,-1,Company - Private,-1,-1,Unknown / Non-Applicable


In [207]:
df_no_dup = df.drop_duplicates()

In [208]:
df_no_dup.to_csv('glassdoor_ds_job_salary.csv', index=False)

In [16]:
newly_scraped

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist V,"$111,371 /yr (est.)","Posting End Date:\nApril 09, 2023\nEmployee Ty...",4.1,Enbridge\n4.1,"Houston, TX",10000+ Employees,1949,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",$10+ billion (USD)
1,Senior Data Scientist - TS Clearance Required,"$131,400 /yr (est.)",Sr Data Scientist (up to 25% profit sharing be...,-1,Eighty20 Collective,"Sterling, VA",Unknown,-1,Company - Public,-1,-1,Unknown / Non-Applicable
2,Senior Systems Engineer - Systems Modeling & C...,"$125,000 /yr (est.)",You will have an opportunity to directly impac...,4.1,Cenergy International Services\n4.1,"Boston, MA",10000+ Employees,1907,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",$10+ billion (USD)
3,Data Engineer,"$110,000 /yr (est.)",NATO Data Engineer\nRequirements:\nTs/sci or s...,-1,"Bluemont Technology & Research, Inc.","Norfolk, VA",-1,-1,-1,-1,-1,-1
4,Machine Learning Engineer,"$130,000 /yr (est.)",Job Description Overview:\nVectrus seeks a res...,3.9,Vectrus\n3.9,"King George, VA",10000+ Employees,1945,Company - Public,Aerospace & Defense,Aerospace & Defense,Unknown / Non-Applicable
...,...,...,...,...,...,...,...,...,...,...,...,...
145,Data Scientist - RWD,"$150,000 /yr (est.)",Job Summary:\nWe are seeking an experienced Da...,-1,Norstella,Remote,-1,-1,-1,-1,-1,-1
146,Data Scientist,"$120,000 /yr (est.)",Gloo was founded as a mission-driven organizat...,3.6,Gloo\n3.6,"Boulder, CO",51 to 200 Employees,2013,Company - Private,Enterprise Software & Network Solutions,Information Technology,$1 to $5 million (USD)
147,Sr. Data Scientist,"$107,469 /yr (est.)","Become a Part of the NIKE, Inc. Team\nNIKE, In...",4.1,Nike\n4.1,"Beaverton, OR",10000+ Employees,1972,Company - Public,Consumer Product Manufacturing,Manufacturing,$10+ billion (USD)
148,Data Scientist,"$108,421 /yr (est.)","With a focus on Mobility, Operational Excellen...",3.9,Nissan\n3.9,"Franklin, TN",10000+ Employees,1933,Company - Public,Transportation Equipment Manufacturing,Manufacturing,$1 to $5 billion (USD)


After webscraping the data, we put it into a csv file.


DATA CLEANING SECTION

In [39]:
import pandas as pd

In [40]:
df = pd.read_csv('glassdoor_ds_job_salary.csv')

In [41]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist V,"$111,362 /yr (est.)","Posting End Date:\r\nApril 09, 2023\r\nEmployee Type:\r\nRegular-Full time\r\nUnion/Non:\r\nThis is a non-union position\r\nLife takes energy. The Enbridge Technology + Innovation Lab works with data that powers our products to improve safety and reliability. By working hands-on with ground-breaking technology, the lab pioneers the development of innovative products through small agile teams. Our teams incorporate a variety of multidisciplinary skills, including industrial predictive algorithms, machine learning, and sentiment analysis.\r\nAs a Data Scientist V, you’ll bring a deep understanding of big data and will help the T+IL to build and enable big data analytics solutions. We build advanced technology products to deliver energy in a safe and reliable way to fuel a rapidly innovating world. Here in our new Technology + Innovation Lab we believe in Agile way of working, focus on our people and value craftsmanship, in a ‘work is play’ environment where we offer a flexible, fun, and modern working style to support individual needs.",4.1,Enbridge\r\n4.1,"Houston, TX",10000+ Employees,1949,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",$10+ billion (USD)
1,Data Scientist,-1,"Data Scientist\r\nJob Category: Science\r\nTime Type: Full time\r\nMinimum Clearance Required to Start: TS/SCI\r\nEmployee Type: Regular\r\nPercentage of Travel Required: Up to 10%\r\nType of Travel: Local\r\nCACI is seeking a Mathematics / Statistical Scientist (Mid) to perform and manage the production of various products in coordination with project and program leadership. This position will support development and application of modeling and simulation tools and algorithms and data mining, analysis, aggregation, fusion, and visualization to process, analyze, and integrate disparate data sources facilitating the production and dissemination of S&TI and GMI relating to forecasting of future foreign ground forces research, development, weapon systems, employment, and operations. The candidate shall integrate statistical data analysis techniques with raw and finished intelligence and geospatial data and services to produce analysis aligned with research objectives. The work involves ensuring data outputs and analysis are available in actionable, standard, and usable forms and utilizing the application of intelligence principles in concert with a practical knowledge of information technology to provide technical mission support.\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nCompany Overview: At CACI, you will have the opportunity to make an immediate impact by providing information solutions and services in support of national security missions and government transformation for Intelligence, Defense, and Federal Civilian customers. CACI is an Equal Opportunity/Affirmative Action Employer. All qualified applicants will receive consideration for employment without regard to race, color, religion, sex, sexual orientation, gender identity, national origin, disability, status as a protected veteran, or any other protected characteristic.",4.0,CACI\r\n4.0,"Charlottesville, VA",10000+ Employees,1962,Company - Public,Aerospace & Defense,Aerospace & Defense,$1 to $5 billion (USD)
2,Data Science/Analysis Research Librarian,-1,"Hamilton College invites applications for the position of Data Science/Analysis Research Librarian in the Library and Information Technology Services (LITS). The Data Science/Analysis Research Librarian will provide research support and introduce cutting-edge data analysis tools and methodologies to LITS patrons as a member of the Research and Instructional Design (R&ID) Team where research and technology experts work together to support faculty and students with research and teaching. The R&ID Team provides an inclusive, respectful, and responsive service point, works co

In [62]:
#we first check for duplicates
print(f"There are {df.duplicated().sum()} duplicates")

# remove duplicates
df.drop_duplicates(inplace=True)

There are 0 duplicates


C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_69876\2120758634.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [44]:
# check for missing values
df.isna().sum()

Job Title            0
Salary Estimate      0
Job Description      0
Rating               0
Company Name         0
Location             8
Size                 0
Founded              0
Type of ownership    0
Industry             0
Sector               0
Revenue              0
dtype: int64

In [45]:
#remove unnecessary columns
df.drop(['Founded', 'Type of ownership'], axis=1, inplace=True)

We are now going to handle the most important part of hour dataset: salary estimate

In [46]:
#remove all rows that have a missing estimated salary. We need labeled examples in order to build our model!

# We first Count the number of rows with -1 as salary estimate (which means it is missing)
count = df[df['Salary Estimate'] == '-1'].shape[0]
print(f'There are {count} rows with a missing salary estimate')

#remove those rows
df = df[df['Salary Estimate'] != '-1']

There are 57 rows with a missing salary estimate


In [49]:
import re

#Let's check if all the salaries have the same format. For example: $131,400 /yr (est.)


# Define regular expression pattern to check format
pattern = r'\$\d{1,3}(,\d{3})*\s?\/yr\s?\(est.\)'

# Check if all values in the Salary Estimate column have the expected format
matches = df['Salary Estimate'].apply(lambda x: bool(re.match(pattern, x)))

not_matching = df[~matches]
print(f'Number of Vvalues in the Salary Estimate column have do not have the expected format: {len(not_matching)}')
not_matching

Number of Vvalues in the Salary Estimate column have do not have the expected format: 14


Job Title  \
14                     Data Mining Center Journeyman Electrician   
22   Sr Data Analyst-Clinical Evidence Strategy & Implementation   
36               Contract Senior Statistical Programmer (Remote)   
81               AI/ML Autonomous Vehicle Software Video Analyst   
89                                                  Statistician   
98                                Data Scientist_Computer Vision   
134                                               DATA SCIENTIST   
136                                          Senior Statistician   
146                                               Data Scientist   
241                                Senior Statistical Programmer   
250                                               Data Scientist   
279                                          Senior Data Analyst   
300                                               Data Scientist   
402                                               Data Scientist   

        Salary Estimate  \
14    $33.00 /hr (est.)   
22    $35.50 /hr (est.)   
36   $100.00 /hr (est.)   
81    $27.00 /hr (est.)   
89    $36.00 /hr (est.)   
98    $85.00 /hr (est.)   
134   $7,917 /mo (est.)   
136  $115.00 /hr (est.)   
146   $70.00 /hr (est.)   
241   $89.00 /hr (est.)   
250   $76.63 /hr (est.)   
279   $72.50 /hr (est.)   
300   $72.50 /hr (est.)   
402   $72.50 /hr (est.)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [59]:
# We realize that some salaries are expressed at a hourly rate, instead of a yearly rate.


# Identify rows with hourly salary values
hourly_mask = df['Salary Estimate'].str.contains('/hr')

# Convert hourly salary values to yearly salary values
df.loc[hourly_mask, 'Salary Estimate'] = df.loc[hourly_mask, 'Salary Estimate'].apply(lambda x: str(float(x[1:x.index('/')].replace(',', '')) * 2080) + ' /yr (est.)')


# Remove any non-numeric characters and convert to numeric
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda x: re.sub(r'[^0-9]', '', x))

df['Salary Estimate'] = pd.to_numeric(df['Salary Estimate'])

#We now have in our salary estimate column yearly salaries expressed as integers

C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_69876\878150041.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Salary Estimate'] = df['Salary Estimate'].str.replace('$', '').str.replace(',', '').str.replace('/yr (est.)', '')
C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_69876\878150041.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Salary Estimate'] = df['Salary Estimate'].str.replace('$', '').str.replace(',', '').str.replace('/yr (est.)', '')
C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_69876\878150041.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

In [61]:
print(df.dtypes)

Job Title           object
Salary Estimate      int64
Job Description     object
Rating             float64
Company Name        object
Location            object
Size                object
Industry            object
Sector              object
Revenue             object
dtype: object
